In [1]:
%load_ext autoreload
%autoreload 2

import os
os.chdir("..")  # Go up one level to the UROP directory

In [2]:
import torch
from torch.utils.data import DataLoader
import yaml

import librosa
import matplotlib.pyplot as plt
%matplotlib inline

with open("config.yaml") as stream:
    try:
        config = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

batch_size = config["batch_size"]
lr = config["lr"]
num_epochs = config["num_epochs"]
save_every = config["save_every"]
save_dir = config["save_dir"]
num_workers = config["num_workers"]

In [3]:
from utils import LazyWindowedDataset, train_test_split

# Initialize Dataset
full_train_source_dataset = LazyWindowedDataset(
    root_dir="datasets/RoboticArm",
    split="train",
    anomaly_type=['normal'],
    domain_type=['source', 'target'],
    window_size_ms=100,
    stride_ms=50,
)

train_source_dataset, val_source_dataset = train_test_split(full_train_source_dataset)

train_loader = DataLoader(train_source_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers, drop_last=True)
val_loader = DataLoader(val_source_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers, drop_last=True)

In [4]:
test_source_dataset = LazyWindowedDataset(
    root_dir="datasets/RoboticArm",
    split="test",
    anomaly_type=['normal', 'anomaly'],
    domain_type=['source', 'target'],
    window_size_ms=100,
    stride_ms=50,
)
test_loader = DataLoader(test_source_dataset, batch_size=batch_size, shuffle=False, num_workers=1, drop_last=True)

In [5]:
from models import BaselineAutoencoder
from utils import train_model, evaluate, save_model_checkpoint, load_model_checkpoint

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

model, optimizer, start_epoch, _, _, _ = load_model_checkpoint("checkpoints/RoboticArm_BaselineAutoencoder_epoch_5.pt", BaselineAutoencoder, None)
print(f"Loaded model from epoch {start_epoch}")
model.to(device)

criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

Using device: cuda
Loaded model from epoch 5


In [6]:
loss, auc = evaluate(model, test_loader, criterion)
print(f"Initial loss: {loss:.4f}, AUC: {auc:.4f}")

Initial loss: 0.6313, AUC: 0.8760


In [7]:
model, train_losses, val_losses, val_aucs = train_model(
    name="RoboticArm2",
    model=model,
    criterion=criterion,
    optimizer=optimizer,
    train_loader=train_loader,
    val_loader=val_loader,
    start_epoch=start_epoch,
    num_epochs=10,
    save_every=3,
    save_dir=save_dir,
    verbose=False,
)

Epoch [1/10] | Train Loss: 0.583682 | Val Loss: 0.583207 | Val AUC: nan
Time Spent: 547.99s | ETA: 4931.94s | Current Time: 2025-04-23 10:59:23
Epoch [2/10] | Train Loss: 0.581035 | Val Loss: 0.583079 | Val AUC: nan
Time Spent: 548.88s | ETA: 4391.02s | Current Time: 2025-04-23 11:08:32
Epoch [3/10] | Train Loss: 0.578951 | Val Loss: 0.583029 | Val AUC: nan
Time Spent: 547.82s | ETA: 3834.74s | Current Time: 2025-04-23 11:17:40
Checkpoint saved at ./checkpoints/RoboticArm2_BaselineAutoencoder_epoch_8.pt
Epoch [4/10] | Train Loss: 0.577086 | Val Loss: 0.583160 | Val AUC: nan
Time Spent: 545.55s | ETA: 3273.30s | Current Time: 2025-04-23 11:26:46
Epoch [5/10] | Train Loss: 0.575295 | Val Loss: 0.582965 | Val AUC: nan
Time Spent: 547.04s | ETA: 2735.18s | Current Time: 2025-04-23 11:35:53
Epoch [6/10] | Train Loss: 0.573637 | Val Loss: 0.582909 | Val AUC: nan
Time Spent: 545.82s | ETA: 2183.27s | Current Time: 2025-04-23 11:44:59
Checkpoint saved at ./checkpoints/RoboticArm2_BaselineAutoe

KeyboardInterrupt: 

In [8]:
loss, auc = evaluate(model, test_loader, criterion)
print(f"Final loss: {loss:.4f}, AUC: {auc:.4f}")

Final loss: 0.6332, AUC: 0.9000
